In [106]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import re
import collections
from collections import defaultdict, Counter
import pandas as pd
import math
import copy

In [85]:
def train_bayes(train_file):
    file = open(train_file, 'r')

    tweets = file.readlines()
    tweets = [re.sub('\n', '', tweet) for tweet in tweets]

    targets = [tweet.split()[0] for tweet in tweets]
    text = [' '.join(tweet.split()[1:]) for tweet in tweets]

    # learn the counts of locations
    p_L = collections.Counter(targets)

    # learn counts for each word by location
    cv = CountVectorizer()
    X = cv.fit_transform(text)
    word_df = pd.DataFrame(X.toarray(), columns = cv.get_feature_names())
    word_df['loc_target_Neelan'] = targets
    word_df = word_df.groupby(by = 'loc_target_Neelan').sum()
    word_df['loc_counts_Neelan'] = [p_L[key] for key in sorted(p_L.keys())]

    return word_df

In [139]:
top_word_df = train_df.loc[:, (train_df.sum(axis=0) > 5)].astype('double')
top_word_df.drop('loc_counts_Neelan', axis = 1, inplace=True)

In [166]:
top_words = []

for token in top_word_df.columns:
    total_words = top_word_df[token].sum()
    top_word_df[token] = top_word_df[token] / total_words

for loc in set(targets):
    top_words.append((loc, [i for i in top_word_df.loc[loc].nlargest(20).index]))

top_words

[('Washington,_DC',
  ['amend',
   'churchkeydc',
   'ecd',
   'igdc',
   'kidsfirst',
   'mguh',
   'momofuku',
   'scifest',
   'sibley',
   'smithsonian',
   'truckeroo',
   'washingtondc',
   'whitehouse',
   'wpgc',
   'dc',
   'capitol',
   'fbi',
   'washington',
   'georgetown',
   'nationals']),
 ('Orlando,_FL',
  ['32801',
   '32803',
   '32804',
   '32805',
   '32814',
   '911hangup',
   'accidentinjuries',
   'accidentminor',
   'accidentwithroadblockage',
   'amelia',
   'amway',
   'attemptedsuicide',
   'baldwinpark',
   'bumby',
   'centralbusinessdistrict',
   'collegepark',
   'colonial',
   'commercialalarm',
   'conroy',
   'designatedpatrolareaavailable']),
 ('Boston,_MA',
  ['allston',
   'berklee',
   'bidmc',
   'bombings',
   'bos',
   'bostonmarathon2016',
   'dorchester',
   'eastboston',
   'fenway',
   'fenwaypark',
   'mwen',
   'roslindale',
   'roxbury',
   'massachusetts',
   'onebostonday',
   'boston',
   'bostonstrong',
   'illegal',
   'redsox',
   

In [163]:
a = [i for i in top_word_df.loc['Atlanta,_GA'].nlargest(5).index]

In [164]:
a

['285', 'atlfreestuff', 'atltraffic', 'bedowntown', 'buckhead']

In [130]:
train_df

,00,000,00000kt,000mph,001,003,007,008,00am,00pm,...,zume,zuni,zuniga,zurichseeconnections,zwillinger,zz,zzboy,zzztop,zzzzzzz,loc_counts_Neelan
loc_target_Neelan,,,,,,,,,,,,,,,,,,,,,
"Atlanta,_GA",2,1,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1910
"Boston,_MA",1,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1465
"Chicago,_IL",2,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3020
"Houston,_TX",0,49,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,1,0,0,2461
"Los_Angeles,_CA",0,6,4,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,5988
"Manhattan,_NY",2,8,0,0,1,0,0,1,0,1,...,0,0,0,1,1,0,0,0,0,6787
"Orlando,_FL",8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1401
"Philadelphia,_PA",6,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,1,0,0,0,1663
"San_Diego,_CA",1,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1568


In [86]:
def read_test_file(test_file):
    file = open(test_file, 'r')

    tweets = file.readlines()
    tweets = [re.sub('\n', '', tweet) for tweet in tweets]

    targets = [tweet.split()[0] for tweet in tweets]
    text = [' '.join(tweet.split()[1:]) for tweet in tweets]

    return (targets, text)

In [87]:

def test_one_target(test_tweet, target, train_df):
    test_tweet = test_tweet.lower()
    tokenized_tweet = re.split(r'\W', test_tweet)
    score = math.log(train_df.loc[target]['loc_counts_Neelan'] / train_df['loc_counts_Neelan'].sum())
    for token in tokenized_tweet:
        if token in train_df.columns:
            score += math.log((train_df.loc[target][token] + 1) / (train_df.loc[target].sum() + 1))
        else:
            score += math.log(1 / (train_df.loc[target].sum() + 1))
    return (target, score)

In [88]:
def bayes_test(test_tweet, targets, train_df):
    best_score = -1000000000000000
    best_target = ''
    for loc in targets:
        pos_target, pos_score = test_one_target(test_tweet, loc, train_df)
        if pos_score > best_score:
            best_score = pos_score
            best_target = pos_target
    return best_target

In [89]:
train_df = train_bayes('tweets.train.clean.txt')

In [91]:
test_targets, test_text = read_test_file('tweets.test1.clean.txt')

In [100]:
bayes_test('Finally. @ Hamilton: An American Musical ', set(test_targets), train_df)

'Washington,_DC'